### 모델
초기 계획은 catboost, xgboost, tabnet을 이용해서 pseudo labeling을 만들고 pseudo labeling을 가지고 semi-supervised learning을 수행하는 것 이었습니다. 하지만, pseudo label을 함께 이용하면 오히려 성능이 더 떨어졌고, 세 개의 모델 중 가장 성능이 좋았던 catboost의 결과물을 제출 했습니다. (catboost, xgboost, tabnet의 모든 데이터 전처리는 모델에 적합하게 다르게 해주었습니다.)  
참고) catboost 단일 모델 private score 0.6363, pseudo label 후 private score : 0.6605  
적용하기 전 부터 데이터의 특성상 대부분의 변수가 oridnal 한 관계랑 상관 없이, 특정 시간 혹은 특정 장소가 연착에 영향을 미친다고 생각했기 때문에 catboost가 가장 성능이 좋을 것이라고 생각했으며, 실제 적용했을 때도 가장 성능이 좋았습니다.
### 데이터 학습 : w/o optimization, self weight 이용
optimization vs w/o optimization : 본 모델은 optimization 을 적용하지 않은 모델을 사용했으며, 상세 이유는 아래 training 부분에 나와있습니다.  
catboost의 default loss_function이 logloss여서, 따로 설정하지 않고 바로 사용했습니다.  
class imbalance 해결을 위해 training data에서 라벨 별 가중치를 구해서 모델 학습할 때 사용했습니다.  
(catboost의 auto_class_weights parameter가 있는데, True로 놓으면 거의 유사한 방식으로 동작할 것 으로 예상합니다.)

In [ ]:
!pip install catboost==1.1.1

# Import

In [2]:
import random
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from catboost import CatBoostClassifier, Pool
from collections import defaultdict


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Fixed Seed

## (csv를 parquet 형태로 저장하지 않으신 분만 실행하세요)

In [ ]:
# # baseline을 참고했으며, parquet으로 저장하고 실행하면 csv보다 매우 빠르게 데이터 처리가 가능합니다 :)

# def csv_to_parquet(csv_path, save_name):
#     df = pd.read_csv(csv_path)
#     df.to_parquet(f'./{save_name}.parquet')
#     del df
#     gc.collect()
#     print(save_name, 'Done.')
    
# csv_to_parquet('./train.csv', 'train')
# csv_to_parquet('./test.csv', 'test')

# Load

In [3]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
sample_submission = pd.read_csv('./sample_submission.csv', index_col = 0)

# Preprocessing

### 비행기 연착과 관련되는 데이터의 특성을 생각하며 전처리를 해주었습니다. 정답은 없으며 제 insight를 최대한 활용했습니다.

## train info (참고)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 20 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              890985 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         890921 non-null   object 
 13  Distance                  10

## 1. Month, Days 처리

   비행기 연착과 관련있는 특성을 생각해보면, **특정 기념일에 따라 (예를 들어, 크리스마스) 연착이 될 관련성이 가장 크다**고 생각(가정)했습니다. 
    따라서, Month와 Day를 따로 생각하기 보다 이것을 합쳐서 366일(해당 연도가 윤년일 경우 고려)으로 표현했습니다.
    이런 가정 내에서는 날짜의 순서는 전혀 관계없으며 특정 일만 카테고리 형식으로 처리합니다(모델에서).

In [4]:
def to_days(x):
    month_to_days = {1:0, 2:31, 3:60, 4:91, 5:121, 6:152, 7:182, 8:213, 9:244, 10:274, 11:305, 12:335}
    return month_to_days[x]

train.loc[:, 'Day'] = train['Month'].apply(lambda x: to_days(x))
train['Day'] = train['Day'] + train['Day_of_Month']

test.loc[:, 'Day'] = test['Month'].apply(lambda x: to_days(x))
test['Day'] = test['Day'] + test['Day_of_Month']

train = train.astype({'Day':object})
test = test.astype({'Day':object})

print("Day Done.")

Day Done.


## 2. Carrier_ID(DOT) 처리 with Airline

데이터의 설명을 보면 Carrier_ID(DOT)은 해당 항공사(Airline) 별로 하나가 할당 됩니다(1대1 대응). 따라서, 둘 중 하나만 있어도 분류하는데 지장이 없으며(두 데이터는 서로 완전종속이기 때문에), 각각 891080개의 non-null 값을 보면, 어쩌면, 두 개 모두 결측치가 아닌 이상, 데이터 복구가 가능하다고 가정을 할 수 있습니다. **둘 다 상관없지만 저는 숫자로 표현된게 마음이 편해서 Carrier_ID(DOT)을 사용**하기로 마음먹었습니다. 따라서, 먼저 복구 가능한 데이터의 개수 (Airline이 존재하면서, Carrier_ID(DOT)이 존재 안하는 row의 개수)를 확인한 후, Airline과 Carrier_ID(DOT)을 이어주는 dictionary 를 만들어서 가능한 Carrier_ID(DOT)를 복구합니다.

### Airline 을 통해 Carrier_ID(DOT) 복구 가능한 데이터의 개수 파악 및 복구

In [8]:
cond1 = train['Carrier_ID(DOT)'].isnull()
cond2 = ~train['Airline'].isnull()
print("Carrier_ID(DOT) 복구 가능한 데이터의 개수 :", len(train.loc[cond1 & cond2, :]))

Carrier_ID(DOT) 복구 가능한 데이터의 개수 : 97114


### Airline to Carrier_ID(DOT) dictinary 만들기

In [10]:
# airline to carrier id, dictinary 만들기
# 모두 데이터가 존재하는 열에서 Dict[Airline] = carrier_ID(DOT) 가 되도록 dictionary 생성
airline_to_cid = {}
for _, row in train[(~train['Carrier_ID(DOT)'].isnull() & ~train['Airline'].isnull())].iterrows():
    airline_to_cid[row['Airline']] = row['Carrier_ID(DOT)']

### Carrier_ID(DOT) 복구

In [14]:
# 복구하기
def to_cid(x):
    return airline_to_cid[x]

cond1 = train['Carrier_ID(DOT)'].isnull()
cond2 = ~train['Airline'].isnull()
train.loc[cond1&cond2, 'Carrier_ID(DOT)'] = train.loc[cond1&cond2, 'Airline'].apply(lambda x: to_cid(x))

### 복구 안 된 Carrier_ID는 학습에 방해되므로 제거

In [15]:
# 복구 안 된 row 빼기
train = train.dropna(subset=['Carrier_ID(DOT)'], how='any', axis=0)

### Test Data 처리

test data의 경우 복구 안 된 row를 빼 버리면 안됩니다. 복구 안된 row도 예측을 해야되기 때문이죠.
따라서, Airline, Carrier_ID(DOT) 둘 다 없는 row는 Carrier_ID(DOT)에 최빈 값의 Carrier_ID(DOT)을 채워주도록 하고 Airline 으로부터 복구 가능한 Carrier_ID(DOT)은 Training과 마찬가지로 복구해줍니다.

In [16]:
# (Test Data Only)
# Airline, Carrier_Code 둘 다 없으면 최빈 값으로 대체
NaN_col = ['Carrier_ID(DOT)']
cond1 = test['Airline'].isnull()
cond2 = test['Carrier_ID(DOT)'].isnull()

for col in NaN_col:
    mode = test[col].mode()[0]
    test.loc[cond1&cond2, col] = mode

# 나머진 Airline에서 대체
cond1 = test['Carrier_ID(DOT)'].isnull()
cond2 = ~test['Airline'].isnull()
test.loc[cond1&cond2, 'Carrier_ID(DOT)'] = test.loc[cond1&cond2, 'Airline'].apply(lambda x: to_cid(x))

print("Cid Done.")

Cid Done.


## 3. 제거할 Columns 제거

Month, Day_of_Month 는 Day의 데이터로 합쳐졌으니 제거합니다.  
Cancelled, Diverted 는 모두 0이므로 의미없는 값이므로 제거합니다.  
Origin_Airport, Destination_Airport 는 Origin_Airport_ID와 Destination_Airport_ID와 1대1대응 이므로 제거합니다.  
Carrier_Code(IATA)는 Airline 별로 N:1로 할당 된 값이므로, 큰 의미가 없습니다. 제거합니다.  
State 정보 보다는 Origin_Airport_ID가 중요하다고 생각되어서 (미국 주는 너무 커서 의미가 없다고 생각했음) 제거했습니다.

In [17]:
col_drop = ['Month', 'Day_of_Month', 'Cancelled', 'Diverted', 'Origin_Airport', 'Destination_Airport', 'Carrier_Code(IATA)', 'Airline', 'Origin_State', 'Destination_State']
train = train.drop(col_drop, axis=1)
test = test.drop(col_drop, axis=1)
print("Drop Done.")

Drop Done.


## 4. Estimated Departure Time (EDT), Estimated Arrival Time (EAT) 복구

Airline, Carrier_ID(DOT)의 경우와 비슷하게 EDT &rarr; EAT 복구, EAT &rarr; EDT 복구가 가능합니다.  
복구 방법은 **결측치가 없는 Origin_Airport_ID &rarr; Destination_Airport_ID 의 정보를 이용**하는 것 입니다.  
본 데이터를 살펴보신분들은 아시겠지만, EDT보다 EAT가 낮은 경우도 있습니다. (-시차 때문에 발생) 하지만, 이것과 별개로 특정 출발지 &rarr; 도착지의 관계에서는 EAT-EDT가 평균적으로 비슷할 것 입니다. (마이너스인 경우에도 항상 마이너스기 때문에 비슷할 것) 언제 출발이던 인천 &rarr; 오사카의 걸리는 시간은 비슷하기 때문입니다.  
따라서 기존 데이터로부터 Origin_Airport_ID (=OAID), Origin_Departure_ID (=ODID)의 쌍에 대해 걸리는 평균 시간을 구하면 EDT, EAT 둘 중 하나만 알고 있으면 서로 데이터 복구가 가능합니다.  
이 과정에서, 계산을 용이하게 하기 위해 hour:minute의 데이터를 모두 minute으로 변경해주었고 (하루 24시 = 1440분) 이 과정에서 EAT-EDT 했을 때, 음수가 나올 수 있는데 이것을 처리하기 위해 modular 연산을 이용했습니다. 

### Arrival Time & Departure Time 둘 다 60분 * 24 = 1440 계로 바꿔주기

In [18]:
def to_minutes(x):
    x = int(x)
    x = str(x)
    if len(x) > 2:
        hours, mins = int(x[:-2]), int(x[-2:])
    else:
        hours, mins = 0, int(x[-2:])
    return hours*60+mins

estimated_times = ['Estimated_Departure_Time', 'Estimated_Arrival_Time']

for ET in estimated_times:
    cond = ~train[ET].isnull()
    train.loc[cond, ET] = train.loc[cond, ET].apply(lambda x: to_minutes(x))
    cond2 = ~test[ET].isnull()
    test.loc[cond2, ET] = test.loc[cond2, ET].apply(lambda x: to_minutes(x))

### EAT, EDT 모두 없는 값은 Training에 도움이 안 되므로 빼주기

In [20]:
train = train.dropna(subset=['Estimated_Arrival_Time', 'Estimated_Departure_Time'], how ='all', axis=0)

### (OAID, DAID)를 키로 갖고, 평균 비행시간을 값으로 갖는 dictionary 만들기

In [21]:
time_flying = defaultdict(int)
time_number = defaultdict(int)

cond_arr2 = ~train['Estimated_Arrival_Time'].isnull()
cond_dep2 = ~train['Estimated_Departure_Time'].isnull()

for _, row in train.loc[cond_arr2 & cond_dep2, :].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    time_flying[(OAID,DAID)] += (row['Estimated_Arrival_Time'] - row['Estimated_Departure_Time'])%1440 # 하루 최대는 1440분
    time_number[(OAID,DAID)] += 1
    
    
for key in time_flying.keys():
    time_flying[key] /= time_number[key]

### Dictionary를 이용해 EAT, EDT 복구

In [22]:
for index, row in train.loc[train['Estimated_Departure_Time'].isnull(),].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    train.loc[index,'Estimated_Departure_Time'] = \
        (train.loc[index]['Estimated_Arrival_Time'] - time_flying[(OAID, DAID)])%1440
    
for index, row in train.loc[train['Estimated_Arrival_Time'].isnull(),].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    train.loc[index,'Estimated_Arrival_Time'] = \
        (train.loc[index]['Estimated_Departure_Time'] + time_flying[(OAID, DAID)])%1440

### Test Data 처리

test data의 경우 복구 안 된 row를 빼 버리면 안됩니다. 복구 안된 row도 예측을 해야되기 때문이죠.  
따라서, EAT, EDT 둘 다 없는 row는 각각의 최빈값을 채워주도록 하고 나머지는 복구를 해줍니다.

In [23]:
# (Test Data Only)
# 둘 다 없으면 최빈값으로 대체
cond_1 = test['Estimated_Departure_Time'].isnull()
cond_2 = test['Estimated_Arrival_Time'].isnull()

mode = test['Estimated_Departure_Time'].mode()[0]
mode2 = test['Estimated_Arrival_Time'].mode()[0]
test.loc[cond_1&cond_2, ['Estimated_Departure_Time', 'Estimated_Arrival_Time']] = mode, mode2


# Departure만 없을 때,
for index, row in test.loc[test['Estimated_Departure_Time'].isnull(),].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    test.loc[index,'Estimated_Departure_Time'] = \
        (test.loc[index]['Estimated_Arrival_Time'] - time_flying[(OAID, DAID)])%1440
    

# Arrival만 없을 때,
for index, row in test.loc[test['Estimated_Arrival_Time'].isnull(),].iterrows():
    OAID, DAID = row['Origin_Airport_ID'], row['Destination_Airport_ID']
    test.loc[index,'Estimated_Arrival_Time'] = \
        (test.loc[index]['Estimated_Departure_Time'] + time_flying[(OAID, DAID)])%1440

    
# 모두 int로 바꾼다.
estimated_times = ['Estimated_Departure_Time', 'Estimated_Arrival_Time']
train = train.astype({'Estimated_Departure_Time':int, 'Estimated_Arrival_Time':int})
test = test.astype({'Estimated_Departure_Time':int, 'Estimated_Arrival_Time':int})
for ET in estimated_times:
    train.loc[train[ET] == 1440, ET] = 0
    test.loc[test[ET] == 1440, ET] = 0


print("EDT, EAT Done.")

EDT, EAT Done.


## Object화 for CatBoost

### EDT, EAT 처리

**특정 시간**이 비행기 연착에 영향을 미칠 것으로 생각했습니다. 따라서 EDT, EAT경우 30분 단위로 48개의 bin에 담아서 object화 해주었습니다. 그렇게 함으로써 ordinal한 관계는 전혀 없으면서 비슷한 시간대(30분 단위)는 같은 특성을 갖도록 했습니다.

In [24]:
# EDT, EAT 48개의 bins에 담으면 된다. 1440(60*24) 계니까, 48씩 끊어서 하면 될 듯
estimate_times = ['Estimated_Departure_Time', 'Estimated_Arrival_Time']
names = {'Estimated_Departure_Time':'EDT', 'Estimated_Arrival_Time':'EAT'}
for ET in estimated_times:
    for i in range(48):
        train.loc[train[ET].between(i*30, (i+1)*30, 'left'), names[ET]] = i
        test.loc[test[ET].between(i*30, (i+1)*30, 'left'), names[ET]] = i

train = train.astype({'EDT':int, 'EAT':int})
test = test.astype({'EDT':int, 'EAT':int})

train = train.drop(['Estimated_Departure_Time', 'Estimated_Arrival_Time'], axis=1)
test = test.drop(['Estimated_Departure_Time', 'Estimated_Arrival_Time'], axis=1)

print("EDT, EAT Done.")

EDT, EAT Done.


### Distance 및 모든 값 object 처리

Distance는 사실, ordinal하게 상관관계가 있을 것으로 예상되었지만, catboost 특성을 고려하여 object화 해주었습니다. Object화 할 때, 모든 distance에 대해 카테고리를 나누면 너무 많아지기 때문에, 단위를 100개로 총 50개의 bin에 담았습니다.  
나머지 (EDT, EAT, OAID, DAID, Carrier_ID(DOT))의 모든 값은 category가 맞다고 생각하였기 때문에 **(not ordinal이면서 특정 값이 영향을 미침)** 모두 object화 해주었습니다.

In [25]:
for i in range(51):
    train.loc[train['Distance'].between(i*100, (i+1)*100, 'left'), 'Distance'] = i
    test.loc[test['Distance'].between(i*100, (i+1)*100, 'left'), 'Distance'] = i

train = train.astype({'Distance':int})
test = test.astype({'Distance':int})

print("distance Done.")

train = train.astype({'Carrier_ID(DOT)':int})
test = test.astype({'Carrier_ID(DOT)':int})

train = train.astype({'EDT':object, 'EAT':object, 'Distance':object, 'Origin_Airport_ID':object, \
                     'Destination_Airport_ID':object, 'Carrier_ID(DOT)':object})
test = test.astype({'EDT':object, 'EAT':object, 'Distance':object, 'Origin_Airport_ID':object, \
                     'Destination_Airport_ID':object, 'Carrier_ID(DOT)':object})

print("CID Done.")

distance Done.
CID Done.


# Modeling & Training

## Preparation

In [27]:
train = train.dropna()

column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))

train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

print('Training Prepared.')

Training Prepared.


## self weight

전체 데이터에서 delay된 데이터는 not_delay 데이터 보다 매우 적기 때문에 학습할 때, 이를 해결하기 위해 클래스 별 가중치를 계산하여 이것을 고려하여 주어서 학습하도록 했습니다. 이렇게 함으로써, delay, not_delay 데이터의 imbalance를 해결했습니다.

In [28]:
counts = list(train_y.value_counts())
class_weight = [counts[1]/sum(counts), counts[0]/sum(counts)]
print("weight :", class_weight)

weight : [0.1766316896968529, 0.8233683103031472]


## Training

### Optimization 여부 ?

Optimization을 해서 예상한 결과값도 있고, 하지 않은 결과값도 있습니다. 제출 csv는 optimization을 진행하지 않은 코드이기 때문에 이것으로 제출 합니다.
optimization을 해보기도 했고 안 해보기도 했는데, optimization을 하지 않은 코드가 public 성능이 조금 더 좋아서, 이것을 제출했습니다. **(사실 올바른 모델 평가를 위해서는 optimization을 하고 내는게 더 성능이 좋을 것이라고 생각은 합니다.)**
하지만, optimization을 위해서는 validation 세트가 필요하고 data를 0.8 / 0.2 이런식으로 나눠야 합니다. 전체 데이터에 delay 결측치가 많으므로, 최대한 많은 데이터를 보존하기 위해 optimization을 안하고 더 많은 데이터셋으로 학습한 코드를 냈네요.  
따라서 아래와 같이 CatBoost 기본 모델과 self-weight를 사용했습니다.

In [29]:
cat_features = [i for i in range(8)]
model = CatBoostClassifier(random_seed=42, cat_features=cat_features, class_weights=class_weight, verbose=0)
model.fit(train_x, train_y)

## Inference

In [30]:
y_pred = model.predict_proba(test_x)

## Save the prediction

In [31]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('./results/catboost_self_weight2.csv', index=True)

print("CSV Done.")

CSV Done.
